In [1]:
# Keep this running in the command line:
# ssh -NL 12345:localhost:27017 myaws &
# To kill:
# use ps | grep "ssh -NL" to find jobID, then
# kill [jobID]
# 
# Indeed API: https://github.com/indeedlabs/indeed-python

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import json
from pymongo import MongoClient
from indeed import IndeedClient
import pickle
%matplotlib inline

### set up indeed connection

In [6]:
indeedClient = IndeedClient(publisher = 5028694075236965)

### set up mongo connection

In [10]:
mongoClient = MongoClient(port = 12345)

In [11]:
db = mongoClient.indeed

In [12]:
coll = mongoClient.indeedColl

### indeed query

In [73]:
i = 0
params = {
    'q' : "(python or 'data science' or 'data scientist')",
    'l' : "Foster City, CA", # kind of the midpoint of SF to South Bay
    'userip' : "13.56.31.137", # my AWS instance static IP
    'useragent' : "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_8_2)", # this is fake...
    'format' : 'json', # this is the default anyways
    'radius' : 50, # n miles,
    'start' : i, # page num
    'fromage' : 60, # get results from within n days
    'limit' : 25, # display per page, 25 is max, probably doesn't matter
    'latlong' : 1 # If latlong=1, returns latitude and longitude information
}

search_response = indeedClient.search(**params)

In [74]:
search_response.keys()

dict_keys(['version', 'query', 'location', 'paginationPayload', 'radius', 'dupefilter', 'highlight', 'totalResults', 'start', 'end', 'pageNumber', 'results'])

In [82]:
search_response['totalResults']

2815

In [105]:
search_response['results'][14]

{'city': 'Menlo Park',
 'company': 'GRAIL, Inc.',
 'country': 'US',
 'date': 'Fri, 21 Apr 2017 19:25:00 GMT',
 'expired': False,
 'formattedLocation': 'Menlo Park, CA',
 'formattedLocationFull': 'Menlo Park, CA',
 'formattedRelativeTime': '30+ days ago',
 'indeedApply': False,
 'jobkey': '3ca55e0327b8b7a1',
 'jobtitle': 'DATA SCIENTIST',
 'language': 'en',
 'latitude': 37.453297,
 'longitude': -122.18132,
 'onmousedown': "indeed_clk(this,'7542');",
 'snippet': '(e.g., as a statistician / <b>data</b> <b>scientist</b> / computational biologist / bioinformatician). Our <b>data</b> <b>science</b> team is responsible for cleaning, preparing, and...',
 'source': 'GRAIL, Inc.',
 'sponsored': False,
 'state': 'CA',
 'stations': '',
 'url': 'http://www.indeed.com/viewjob?jk=3ca55e0327b8b7a1&qd=4FBHK2sUebDRNvMMiQDczzbHzQ-UZ4C4-oK5eLO8_jT88N2g4AZtysN66g5h9LVhyrqCy1phgtnIprHVf4O82a3yVzE5vdpHnSOwHcA7tdK0bs24pKrFxhqoHah2yOE0kUfBOdYpj1ytmz3gC0lTzHSYkm1y7z4S2QfxOYra11BwpeO1BCqxKFMqdiiXWteu&indpubnum=5

In [106]:
jobkey_li = ["3ca55e0327b8b7a1"]
job_response = indeedClient.jobs(jobkeys = jobkey_li)

In [87]:
job_response.keys()

dict_keys(['version', 'results'])

In [110]:
job_response['results'][0]

{'city': 'Menlo Park',
 'company': 'GRAIL, Inc.',
 'country': 'US',
 'date': 'Fri, 21 Apr 2017 19:25:00 GMT',
 'expired': False,
 'formattedLocation': 'Menlo Park, CA',
 'formattedLocationFull': 'Menlo Park, CA',
 'formattedRelativeTime': '30+ days ago',
 'indeedApply': False,
 'jobkey': '3ca55e0327b8b7a1',
 'jobtitle': 'DATA SCIENTIST',
 'language': 'en',
 'latitude': 37.453297,
 'longitude': -122.18132,
 'onmousedown': "indeed_clk(this,'');",
 'recommendations': [],
 'snippet': 'PhD degree in a quantitative discipline (e.g., statistics, computational biology, computer science, mathematics, physics, electrical engineering). 2+ years of relevant work experience in data analysis or related field. Conduct end-to-end analysis that include design, data gathering, processing, analysis, iteration with stakeholders, and dissemination of results....',
 'source': 'GRAIL, Inc.',
 'sponsored': False,
 'state': 'CA',
 'stations': '',
 'url': 'http://www.indeed.com/rc/clk?jk=3ca55e0327b8b7a1&atk='}

In [97]:
job_response['results'][0].keys()

dict_keys(['jobtitle', 'company', 'city', 'state', 'country', 'language', 'formattedLocation', 'source', 'date', 'snippet', 'url', 'onmousedown', 'latitude', 'longitude', 'jobkey', 'sponsored', 'expired', 'indeedApply', 'formattedLocationFull', 'formattedRelativeTime', 'stations', 'recommendations'])

In [98]:
search_response['results'][4].keys()

dict_keys(['jobtitle', 'company', 'city', 'state', 'country', 'language', 'formattedLocation', 'source', 'date', 'snippet', 'url', 'onmousedown', 'latitude', 'longitude', 'jobkey', 'sponsored', 'expired', 'indeedApply', 'formattedLocationFull', 'formattedRelativeTime', 'stations'])

In [108]:
len(job_response['results'][0]['snippet'])

367

In [109]:
print(job_response['results'][0]['snippet'])

PhD degree in a quantitative discipline (e.g., statistics, computational biology, computer science, mathematics, physics, electrical engineering). 2+ years of relevant work experience in data analysis or related field. Conduct end-to-end analysis that include design, data gathering, processing, analysis, iteration with stakeholders, and dissemination of results....


### populate mongo

In [45]:
search_response.next()

AttributeError: 'dict' object has no attribute 'next'